# Infancias, vegetación y proximidad

This notebook plots population of the ages 0-5 (early childhood), NDVI vegetation categories and proximity analysis for a city.

In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from pandas.api.types import CategoricalDtype

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/usr/local/python/lib/python3.10/site-packages/osmnx/utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


## Config notebook

In [2]:
# City of analysis
city = 'Monterrey'

# Resolution of analysis
# (Resolutions 8,9,10,11 available for NDVI but pop0a5 only available in hex_bins_pop_2020, res=8)
res = 8
#! Name of proximity analysis table must be changed manually.

# Save plots to output?
save = True
save_dir = "../../../output/figures/ndvi_infancia/"

## Download data

### Download data - Proximity analysis

In [3]:
prox_schema = 'prox_analysis'
prox_table = 'proximityanalysis_20_ageb_hex8'

query = f"SELECT * FROM {prox_schema}.{prox_table} WHERE \"city\" LIKE \'{city}\'"
prox_gdf = aup.gdf_from_query(query, geometry_col='geometry')

# Show
print(prox_gdf.shape)
prox_gdf.head(2)

(1545, 34)


,hex_id,res,geometry,max_escuelas,max_preescolar,max_primaria,max_secundaria,max_servicios comunitarios,max_salud,max_guarderías,...,pobmas,pob_0a2,pob_3a5,pob_0a5,pob_0a14,pob_15a24,pob_25a59,p_60ymas,dens_pobha,city
0,8848a23881fffff,8,"POLYGON ((-100.38026 25.93877, -100.38069 25.9...",24.460722,24.460722,21.807982,9.385404,44.928272,22.735261,44.928272,...,123.34427,14.639343,11.836064,26.475407,65.098359,43.606558,109.950839,31.459015,2.970079,Monterrey
1,8848a23883fffff,8,"POLYGON ((-100.38588 25.94625, -100.38631 25.9...",14.182725,14.182725,12.408625,8.665118,60.525388,12.251763,60.525388,...,234.69318,26.695550,25.030443,51.725993,122.381728,78.758787,219.166333,65.829040,5.674415,Monterrey


### Download data - NDVI Data

In [4]:
ndvi_schema = 'raster_analysis'
ndvi_table = 'ndvi_analysis_hex'

query = f"SELECT * FROM {ndvi_schema}.{ndvi_table} WHERE \"city\" LIKE \'{city}\' AND \"res\"={res}"
ndvi_gdf = aup.gdf_from_query(query, geometry_col='geometry')

# Show
print(ndvi_gdf.shape)
ndvi_gdf.head(2)

(1545, 11)


,hex_id,geometry,res,ndvi_mean,ndvi_std,ndvi_median,ndvi_max,ndvi_min,ndvi_diff,ndvi_tend,city
0,8848a20021fffff,"POLYGON ((-100.19993 25.62874, -100.19476 25.6...",8,0.315942,0.141914,0.322969,0.454254,0.059041,0.395213,-0.002236,Monterrey
1,8848a20023fffff,"POLYGON ((-100.20595 25.64143, -100.20553 25.6...",8,0.266513,0.118887,0.271825,0.387413,0.048207,0.339206,-0.002337,Monterrey


### Download data - 2020 Census data 

In [5]:
mun_schema = 'metropolis'
mun_table = 'metro_gdf'

query = f"SELECT * FROM {mun_schema}.{mun_table} WHERE \"city\" LIKE \'{city}\'"
mun_gdf = aup.gdf_from_query(query, geometry_col='geometry')

# Show
print(mun_gdf.shape)
mun_gdf.head(2)

(18, 6)


,CVEGEO,CVE_ENT,CVE_MUN,NOMGEO,geometry,city
0,19001,19,001,Abasolo,"POLYGON ((-100.39267 25.98715, -100.39224 25.9...",Monterrey
1,19006,19,006,Apodaca,"POLYGON ((-100.24794 25.86462, -100.24672 25.8...",Monterrey


In [ ]:
# Get cvegeos of city
cvegeo_list = list(mun_gdf.loc[mun_gdf.city==city]["CVEGEO"].unique())

# Donwload hexagons with pop data
hex_pop = gpd.GeoDataFrame()
hex_schema = 'censo'
hex_table = 'hex_bins_pop_2020'

# Iterate over municipality codes for each metropolitan area or capital
for cvegeo in cvegeo_list:
    # Downloads municipality polygon according to code
    query = f"SELECT * FROM {hex_schema}.{hex_table} WHERE \"CVEGEO\" LIKE \'{cvegeo}%%\'"
    hex_tmp = aup.gdf_from_query(query, geometry_col='geometry')
    hex_pop = pd.concat([hex_pop, hex_tmp],
    ignore_index = True, axis = 0)
    
# Show
print(hex_pop.shape)
hex_pop.head(2)

## Data treatment

### Data treatment - Early childhood % Calculation

In [ ]:
# Calculate
hex_pop['p_infancia'] = hex_pop['p_0a2'] + hex_pop['p_3a5']
hex_pop['pct_infancia'] = hex_pop['p_infancia'] / hex_pop['pobtot']

# Show
hex_pop.plot('pct_infancia', legend=True)

### Data treatment - Childhood and NDVI Categories 

Early childhood categories

In [ ]:
hex_pop['pct_inf_rango'] = np.nan
hex_pop.loc[hex_pop.pct_infancia>=0.14 , 'pct_inf_rango'] = '+14%'
hex_pop.loc[(hex_pop.pct_infancia>=0.12 )&
             (hex_pop.pct_infancia<0.14), 'pct_inf_rango'] = '12% - 14%'
hex_pop.loc[(hex_pop.pct_infancia>=0.10)&
             (hex_pop.pct_infancia<0.12), 'pct_inf_rango'] = '10% - 12%'
hex_pop.loc[(hex_pop.pct_infancia>=0.08)&
             (hex_pop.pct_infancia<0.1), 'pct_inf_rango'] = '8% - 10%'
hex_pop.loc[(hex_pop.pct_infancia>=0.06)&
             (hex_pop.pct_infancia<0.08), 'pct_inf_rango'] = '6% - 8%'
hex_pop.loc[(hex_pop.pct_infancia>=0.03)&
             (hex_pop.pct_infancia<0.06), 'pct_inf_rango'] = '3% - 6%'
hex_pop.loc[(hex_pop.pct_infancia<0.03), 'pct_inf_rango'] = '0% - 3%'

#Definir orden y convertir pct_inf_rango en categoría ordenada
categories = ['0% - 3%', '3% - 6%', '6% - 8%', '8% - 10%', '10% - 12%', '12% - 14%', '+14%']
hex_pop['pct_inf_rango'] = pd.Categorical(hex_pop['pct_inf_rango'], categories=categories, ordered=True)

NDVI categories

In [ ]:
ndvi_gdf['ndvi_median_rango'] = np.nan
ndvi_gdf.loc[ndvi_gdf.ndvi_median>=0.6 , 'ndvi_median_rango'] = 'Alta densidad vegetal'
ndvi_gdf.loc[(ndvi_gdf.ndvi_median>=0.4 )&
             (ndvi_gdf.ndvi_median<0.6), 'ndvi_median_rango'] = 'Moderada densidad vegetal'
ndvi_gdf.loc[(ndvi_gdf.ndvi_median>=0.2)&
             (ndvi_gdf.ndvi_median<0.4), 'ndvi_median_rango'] = 'Mínima densidad vegetal'
ndvi_gdf.loc[(ndvi_gdf.ndvi_median>=0.1)&
             (ndvi_gdf.ndvi_median<0.2), 'ndvi_median_rango'] = 'Suelo'
ndvi_gdf.loc[(ndvi_gdf.ndvi_median<0.1), 'ndvi_median_rango'] = 'Suelo artificial/Agua/Piedra'


#Definir orden y convertir pct_inf_rango en categoría ordenada
categories = ['Suelo artificial/Agua/Piedra', 'Suelo', 'Mínima densidad vegetal', 'Moderada densidad vegetal', 'Alta densidad vegetal']
ndvi_gdf['ndvi_median_rango'] = pd.Categorical(ndvi_gdf['ndvi_median_rango'], categories=categories, ordered=True)

Proximity analysis categories

In [ ]:
prox_gdf['prox_cat'] = np.nan
prox_gdf.loc[prox_gdf.max_idx_15_min>=60 , 'prox_cat'] = '>60'
prox_gdf.loc[(prox_gdf.max_idx_15_min>=45 )&
             (prox_gdf.max_idx_15_min<60), 'prox_cat'] = '45-60'
prox_gdf.loc[(prox_gdf.max_idx_15_min>=30)&
             (prox_gdf.max_idx_15_min<45), 'prox_cat'] = '30-45'
prox_gdf.loc[(prox_gdf.max_idx_15_min>=15)&
             (prox_gdf.max_idx_15_min<30), 'prox_cat'] = '15-30'
prox_gdf.loc[(prox_gdf.max_idx_15_min<15), 'prox_cat'] = '<15'


#Definir orden y convertir pct_inf_rango en categoría ordenada
categories = ['<15', '15-30', '30-45', '45-60', '>60']
prox_gdf['prox_cat'] = pd.Categorical(prox_gdf['prox_cat'], categories=categories, ordered=True)

### Data treatment - Filter analysis data

In [ ]:
# Filter for ranges of interest
prox_interest_gdf = prox_gdf.loc[(prox_gdf['prox_cat'] == '<15') | 
                                 (prox_gdf['prox_cat'] == '15-30')]

#Refinir orden y de las categorías disponibles para que no haya problemas en la simbología al plotear
categories = ['15-30','<15']
prox_interest_gdf['prox_cat'] = pd.Categorical(prox_interest_gdf['prox_cat'], categories=categories, ordered=True)

# Dissolve data
prox_interest_gdf.to_crs("EPSG:6372",inplace=True)
dissolved = prox_interest_gdf.dissolve('prox_cat')
dissolved.reset_index(inplace=True)
dissolved.to_crs("EPSG:4326",inplace=True)

### Plot data visualization

In [ ]:
# Preparar colores
colors = ["black","orange"]

# Plotear gráficos
fig, ax = plt.subplots(1, 2, figsize=(10, 10))
hex_pop.plot('pct_inf_rango', 'BuPu', ax=ax[0], legend=True,zorder=1)
dissolved.plot('prox_cat',ax=ax[0],zorder=2,
               edgecolor=colors,
               facecolor="none",
               lw=1)

ndvi_gdf.plot('ndvi_median_rango','YlGn', ax=ax[1], legend=True,zorder=1)
dissolved.plot('prox_cat',ax=ax[1],zorder=2,
               edgecolor=colors,
               facecolor="none",
               lw=1)

ax[0].set_title('Porcentaje de población en edad de 0 - 5 años')
ax[1].set_title('Índice de vegetación')

# Guardar gráficos
if save:
    plt.savefig(f"../../../output/figures/ndvi_infancia/ndvi_infancia_prox_{city}.svg")